In [38]:
import numpy as np

from itertools import product

In [ ]:
"""
In-place Pauli {Z,X,Y} measurement on the two slices of a qubit.
Modifies plus_slice (for |0⟩ component) and minus_slice (for |1⟩ component) in place.

Docstring convention from original: Returns 0 for +1 outcome (‘0’), 1 for −1 (‘1’).
NOTE: The original code (and this revised version, to match behavior)
has a specific convention for the 'Y' operator's returned bit:
- Outcome 0 is associated with projection onto a state proportional to |0> - i*|1> (the -1 eigenstate |-i>).
- Outcome 1 is associated with projection onto a state proportional to |0> + i*|1> (the +1 eigenstate |+i>).
This means for 'Y', outcome 0 corresponds to eigenvalue -1, and outcome 1 to eigenvalue +1,
which is inverted compared to the Z and X operators' outcome/eigenvalue mapping.

This version also corrects a bug in the original 'X' and 'Y' probability
calculations (must use np.abs(amplitude)**2 for complex amplitudes).
"""

In [27]:
def measure_qubit(plus_slice, minus_slice, op, rng):

    if op not in ('I', 'Z', 'X', 'Y'):
        raise ValueError(f"Unsupported Pauli operator: {op}. Expected 'I', 'X', 'Y', or 'Z'.")

    if op == 'I':
        # the identity operator does not overwrite the slices and thus lets the state unchanged
        # we return 0 for the +1 eigenvalue as an arbitrary convention
        return 0

    INV_SQRT2 = 1.0 / np.sqrt(2.0)

    plus_slice_tmp = plus_slice.copy()
    minus_slice_tmp = minus_slice.copy()

    # initialize with the Z basis
    plus_ampl = plus_slice_tmp
    minus_ampl = minus_slice_tmp

    plus_eigvec = (1.0, 0.0)
    minus_eigvec = (0.0, 1.0)


    if op == 'X':
        plus_ampl = (plus_slice_tmp + minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp - minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, INV_SQRT2)
        minus_eigvec = (INV_SQRT2, -INV_SQRT2)


    if op == 'Y':
        plus_ampl = (plus_slice_tmp - 1j * minus_slice_tmp) * INV_SQRT2
        minus_ampl = (plus_slice_tmp + 1j * minus_slice_tmp) * INV_SQRT2

        plus_eigvec = (INV_SQRT2, 1j * INV_SQRT2)
        minus_eigvec = (INV_SQRT2, -1j * INV_SQRT2)


    # we square up all amplitudes where we have 0 or 1 at the current index and sum them up
    plus_prob = np.sum(np.abs(plus_ampl)**2)
    plus_prob = np.clip(plus_prob, 0.0, 1.0)
    minus_prob = 1.0 - plus_prob
    plus_meas = rng.random() < plus_prob


    chosen_ampl = (plus_ampl if plus_meas else minus_ampl)
    chosen_eigvec = (plus_eigvec if plus_meas else minus_eigvec)
    renorm_factor = np.sqrt(plus_prob if plus_meas else minus_prob)


    # in case the is tiny, we need to do some special handling
    # renorm_factor == 0 implies chosen_ampl is an array of zeros (assuming a valid normalized initial state).
    if renorm_factor < 1e-12:
        plus_slice[:] = 0.0
        minus_slice[:] = 0.0

        return 0 if plus_meas else 1


    plus_slice[:] = (chosen_ampl * chosen_eigvec[0]) / renorm_factor
    minus_slice[:] = (chosen_ampl * chosen_eigvec[1]) / renorm_factor
    return 0 if plus_meas else 1


def measure_state(state_vec, pauli_ops, rng):
    num_qubits = len(pauli_ops)

    # reshape the state vector in place to allow for multi-dimensional indexing
    state_tensor = state_vec.reshape((2,) * num_qubits)
    outcome_bits = []

    for qubit_idx, pauli_op in enumerate(pauli_ops):
        # we create only a view of the state tensor to be modified in place by the measure_one function
        plus_slice  = state_tensor[(slice(None),) * qubit_idx + (0,)]
        minus_slice = state_tensor[(slice(None),) * qubit_idx + (1,)]

        outcome_bit = measure_qubit(plus_slice, minus_slice, pauli_op, rng)
        outcome_bits.append(outcome_bit)

    return outcome_bits

In [33]:
class PauliOperator:
    def __init__(self, matrix, eigenvalues, eigenvectors):
        self.matrix = matrix
        self.eigenvalues = eigenvalues
        self.eigenvectors = eigenvectors


SQRT2 = np.sqrt(2.0)
sigma_0 = PauliOperator(np.eye(2), [1,1], [np.array([1,0]), np.array([0,1])])
sigma_x = PauliOperator(np.array([[0,1],[1,0]]), [1,-1], [np.array([1,1])/SQRT2, np.array([1,-1])/SQRT2])
sigma_y = PauliOperator(np.array([[0,-1j],[1j,0]]), [1,-1], [np.array([1,1j])/SQRT2, np.array([1,-1j])/SQRT2])
sigma_z = PauliOperator(np.array([[1,0],[0,-1]]), [1,-1], [np.array([1,0]), np.array([0,1])])

pauli_operators = { 'I': sigma_0, 'X': sigma_x, 'Y': sigma_y, 'Z': sigma_z }

In [34]:
def get_theo_exp(state_vec, pauli_ops):
    ops = [pauli_operators[p] for p in pauli_ops]
    vals, vecs = [], []
    for addr in product([0,1], repeat=len(ops)):
        e, v = 1.0, None
        for op, a in zip(ops, addr):
            e *= op.eigenvalues[a]
            v = op.eigenvectors[a] if v is None else np.kron(v, op.eigenvectors[a])
        vals.append(e)
        vecs.append(v)

    probs = [abs(np.vdot(v, state_vec))**2 for v in vecs]
    return sum(e*p for e,p in zip(vals, probs))


def get_obs_exp(state_vec, pauli_ops, num_samples=1000, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    acc = 0.0
    for _ in range(num_samples):
        state_vec_copy = state_vec.copy()           # important, since we modify it in measure_state
        outcome_bits = measure_state(state_vec_copy, pauli_ops, rng)
        acc += (-1) ** np.sum(outcome_bits)
    return acc / num_samples

In [35]:
# construct the state

num_qubits = 10

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
psi = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    psi[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)

In [36]:
# let's use the complex state from above and reconstruct its density matrix


test_measurements = [
    ('Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z'),
    ('X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y'),
    ('Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z'),
    ('X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X'),
    ('Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('X', 'X', 'X', 'X', 'X', 'Y', 'X', 'X', 'X', 'X')
]

num_samples = 1000


print("Performing test measurements...")
theo_exp_dict = {}
obs_exp_dict = {}

for pauli_labels in test_measurements:
    theo_exp = get_theo_exp(psi, pauli_labels)
    obs_exp = get_obs_exp(psi, pauli_labels, num_samples=num_samples)
    theo_exp_dict[pauli_labels] = theo_exp
    obs_exp_dict[pauli_labels] = obs_exp

print("Measurements completed.")

Performing test measurements...
Measurements completed.


In [37]:
print(f"Pauli String | Theoretical Exp | Observed Exp   | Abs. Difference")
print("-" * 70)

differences = []
for pauli_labels_tuple in test_measurements:
    if pauli_labels_tuple in theo_exp_dict and pauli_labels_tuple in obs_exp_dict:
        theo_val = theo_exp_dict[pauli_labels_tuple]
        obs_val = obs_exp_dict[pauli_labels_tuple]
        diff = np.abs(theo_val - obs_val)
        differences.append(diff)

        pauli_str = "".join(pauli_labels_tuple)
        print(f"{pauli_str}   | {theo_val:15.6f} | {obs_val:14.6f} | {diff:15.6f}")


max_diff = np.max(differences)
avg_diff = np.mean(differences)
std_diff = np.std(differences)


print(f"\nNumber of samples per setting: {num_samples}")
print(f"Maximum absolute difference:   {max_diff:.6f}")
print(f"Average absolute difference:   {avg_diff:.6f}")
print(f"Std. Dev. of differences:      {std_diff:.6f}")

Pauli String | Theoretical Exp | Observed Exp   | Abs. Difference
----------------------------------------------------------------------
ZZZZZZZZZZ   |       -1.000000 |      -1.000000 |        0.000000
XXXXXXXXXX   |       -0.000000 |       0.020000 |        0.020000
YYYYYYYYYY   |       -0.000000 |       0.002000 |        0.002000
ZXYZXYZXYZ   |       -0.000000 |       0.010000 |        0.010000
XYZXYZXYZX   |        0.000000 |      -0.004000 |        0.004000
ZZXXXXXXXX   |       -0.000000 |      -0.062000 |        0.062000
ZXXXXXXXXX   |       -0.000000 |       0.044000 |        0.044000
XXXXXYXXXX   |        0.000000 |      -0.066000 |        0.066000

Number of samples per setting: 1000
Maximum absolute difference:   0.066000
Average absolute difference:   0.026000
Std. Dev. of differences:      0.025612
